In [1]:
import os
import time

import numpy as np
from scipy.special import softmax
from tqdm import tqdm
from onnxruntime import (
    GraphOptimizationLevel, InferenceSession,
    SessionOptions, get_all_providers
)
from transformers import GPT2Tokenizer


def create_model_for_provider(
    model_path: str,
    provider: str = 'CPUExecutionProvider'
) -> InferenceSession:
    assert provider in get_all_providers(), \
        f"provider {provider} not found, {get_all_providers()}"
    # Few properties that might have an impact on performances (provided by MS)
    options = SessionOptions()
    options.intra_op_num_threads = int(os.environ.get('NUM_THREADS', 16))
    options.graph_optimization_level = GraphOptimizationLevel.ORT_ENABLE_ALL
    # Load the model as a graph and prepare the CPU backend
    session = InferenceSession(model_path, options, providers=[provider])
    session.disable_fallback()
    return session

In [2]:
tokenizer = GPT2Tokenizer.from_pretrained('./tokenizer')
model = create_model_for_provider('./onnxq/model.onnx')
kv_cache_start = np.load('past_key_values.npy')

In [4]:
def generate(
    text,
    max_len = 100,
    temperature = 1.0,
    top_p = 0.95,
    top_k = 50,
    eod=None,
    additional_eod=[],
    ban = []
):
    if eod is None:
        eod = [50256]
    input_ids = tokenizer(text)['input_ids']
    ids = []
    # kv_cache = np.zeros([30, 2, 1, 32, 1, 96]).astype(np.float32)
    kv_cache = kv_cache_start

    with tqdm() as pbar:
        for i in range(max_len):
            pbar.update(1)
            if i == 0:
                logits, kv_cache = model.run(['output', 'pkv_output'], {
                    "input": np.array([input_ids]).astype(np.int64),
                    'pkv': kv_cache
                })
            else:
                logits, kv_cache = model.run(['output', 'pkv_output'], {
                    "input": np.array([[next_token]], dtype=np.int64),
                    'pkv': kv_cache,
                })

            for x in ban:
                logits[:, -1, x] = -9999

            logits = logits / temperature
            scores = softmax(logits[:, -1, :])
            next_probs = np.sort(scores)[:, ::-1]
            if top_p > 0.0 and top_p < 1.0:
                next_probs = next_probs[:, :int(next_probs.shape[1] * (1 - top_p))]
            if top_k > 0 and top_k < next_probs.shape[1]:
                next_probs = next_probs[:, :top_k]
            next_probs_1 = next_probs / next_probs.sum(axis=1).reshape((-1, 1))

            next_tokens = np.argsort(scores)[:, ::-1]
            if top_p > 0.0 and top_p < 1.0:
                next_tokens = next_tokens[:, :int(next_tokens.shape[1] * (1 - top_p))]
            if top_k > 0 and top_k < next_tokens.shape[1]:
                next_tokens = next_tokens[:, :top_k]

            next_token = np.random.choice(next_tokens[0], p=next_probs_1[0])
            if eod is not None:
                if next_token in eod or next_token in additional_eod:
                    break
            ids.append(next_token)
    return tokenizer.decode(ids)

In [8]:
text = '''这一日,同福客栈的人都坐在了一起,开始讨论下一步怎么办。
佟掌柜:最近生意不好,大家给出出主意?
白展堂:'''
print(generate(text, max_len=100))

100it [00:22,  4.36it/s]

只有请教殷先诚,再作众筹考虑?
宋贤钢也是一副严肃又期待的样子...…
博天堂娱乐主页我在这国际没有直接


In [9]:
text = '''机器助理是一个非常聪明的，智能的机器人，它可以跟你聊天。
用户：你是谁啊？
机器助理：我是机器人，我来自deepdialog，你好。
用户：你多大了？
机器助理：'''
print(generate(text, max_len=100))

100it [00:29,  3.42it/s]

我是一个三周岁的孩子，你还认识我吗？
你好，我是罗永浩。
来源：网络]一年前，在2017年金立M8发布后的一年�


In [10]:
text = '''对联
上联：东南西北；下联：春夏秋冬。
上联：春回大地；下联：福满人间。
上联：云无心以出岫；下联：鸟倦飞而知还。
上联：万事平安幸福年；下联：吉祥如意拜年顺。
上联：人和家顺百事兴；下联：富贵平安福满堂。
上联：新春福旺鸿运开；下联：佳节吉祥如意来。
上联：日子红火喜迎门；下联：天随人意福星照。
上联：人逢喜事精神爽；下联：'''
print(generate(text, max_len=100))

100it [00:56,  1.78it/s]

春风得意笑开颜！
上联：空蒙阴云丽人面；
下联：蓝天丽日新江南。
上联：玉树琼枝瑞草滇�


In [11]:
text = '''写出文本中的关键词：
11月26日上午，市十五届人大常委会第三十五次会议表决通过了关于修改《北京市人口与计划生育条例》的决定，自公布之日起施行。修改后的条例取消了限制生育的措施，明确一对夫妻可以生育三个子女。女方除国家规定的产假外，享受的延长生育假由三十天增加至六十天。同时，子女满三周岁前，夫妻每人每年可享受五个工作日的育儿假。今年5月31日，中共中央政治局召开会议，通过《关于优化生育政策促进人口长期均衡发展的决定》，明确实施一对夫妻可以生育三个子女政策及配套支持措施。8月20日，全国人大常委会审议通过《关于修改<中华人民共和国人口与计划生育法>的决定》。为贯彻落实国家新的生育政策，细化上位法相关制度，本市将修订《北京市人口与计划生育条例》增补为11月市人大常委会会议审议项目。修改后的条例围绕实施三孩生育政策，降低生育养育负担，提振生育水平，同时强化了对全面两孩政策实施前实行计划生育家庭的保障。
关键词：'''
print(generate(text, max_len=100))

100it [01:59,  1.20s/it]

一对夫妻生育第三个子女生育三孩政策细化。
一对夫妻只生育一个子女的，提供经县级以上（含县级）医院鉴�


In [12]:
text = '''将下面文字缩写为摘要：
11月26日上午，市十五届人大常委会第三十五次会议表决通过了关于修改《北京市人口与计划生育条例》的决定，自公布之日起施行。修改后的条例取消了限制生育的措施，明确一对夫妻可以生育三个子女。女方除国家规定的产假外，享受的延长生育假由三十天增加至六十天。同时，子女满三周岁前，夫妻每人每年可享受五个工作日的育儿假。今年5月31日，中共中央政治局召开会议，通过《关于优化生育政策促进人口长期均衡发展的决定》，明确实施一对夫妻可以生育三个子女政策及配套支持措施。8月20日，全国人大常委会审议通过《关于修改<中华人民共和国人口与计划生育法>的决定》。为贯彻落实国家新的生育政策，细化上位法相关制度，本市将修订《北京市人口与计划生育条例》增补为11月市人大常委会会议审议项目。修改后的条例围绕实施三孩生育政策，降低生育养育负担，提振生育水平，同时强化了对全面两孩政策实施前实行计划生育家庭的保障。
摘要：'''
print(generate(text, max_len=100))

100it [02:02,  1.23s/it]

本案涉及未成年人在网络交易中的防范问题，贷款平台用不当方式逃避代还债务并非明智之举，应严厉打�


In [13]:
text = '''文本分类：
基本上可以说是诈骗
选项：积极，消极
答案：'''
print(generate(text, max_len=100))

100it [00:18,  5.53it/s]


以诈骗资金的全部或者部分
截至该公告日收盘价格计算，金花股份主营业务以制冷剂为主营业务，
�


In [15]:
text = '''问题：郑州是哪个省的
答案：河南省
华盛顿属于哪个国家？
答案：美国
中国的首都在哪里？
答案：'''
print(generate(text, max_len=100))

100it [00:22,  4.39it/s]

中国首都北京。
终于被中国同胞的诚挚热情打动，从机场的
安检到登机口的
秘密检查，到飞机上的
严�


In [16]:
text = '''翻译成英文：
不过他承认，美国与欧洲关系密切。
'''
print(generate(text, max_len=100))

100it [00:16,  6.04it/s]

这就向我们展示了一种‘美欧日近心锁’的形态。
这就是美元霸权的根本问题，即‘美元帝国’跨大西�


In [26]:
text = '''翻译成中文：
I would like to have a lunch.
我想去吃午饭。
I love you.
我爱你。
Will you go home with me?
'''
print(generate(text, max_len=100))

100it [00:16,  5.98it/s]

我希望你能回来...）
深圳力邦建筑工程设备股份有限公司是消防行业的龙头标杆企业，致力于为经济


In [18]:
text = '''推理关系判断：
前提：新的权利已经足够好了
假设：每个人都很喜欢最新的福利
选项：矛盾，蕴含，中立
答案：'''
print(generate(text, max_len=100))

100it [00:23,  4.20it/s]

主体：（1）公司法人：（2）股份分配机会：（3）推理关系：
四、新的权利（产婆）新的权利（


In [20]:
text = '''菜谱:西红柿炒鸡蛋
需要原材料:西红柿一个,鸡蛋一个
做法:'''
print(generate(text, max_len=100))

100it [00:17,  5.66it/s]

1.原料:西红柿炒鸡蛋。
2.加足量清水,西红柿切丁。
清水再多一些,泡4小时.待鸡蛋炒熟
原料:�


In [21]:
text = '''关于爱情，乔布斯曾经说过：“'''
print(generate(text, max_len=100))

100it [00:13,  7.26it/s]

爱就在前面，而梦的最初部分，就是痛苦。”
可能你在任何职业生涯中，都遇到过这样的梦魇，但它却�


In [22]:
text = '''根据题目，完成作文
《我和我的母亲》
我的母亲是'''
print(generate(text, max_len=100))

100it [00:15,  6.34it/s]

一个善良的妈妈，乐于助人的好孩子，她能够关心我，照顾我的生活。
妈妈有很多优点，可以说是博观而约取


In [23]:
text = '''望着那几乎成了能量源头的萧炎，薰儿略微有些惊喜，悄悄的退后了一些距离，警戒的守在周围，此时若是将萧炎从这种修炼状态惊醒，恐怕他又将会失去一次晋级的好机会。'''
print(generate(text, max_len=100))

100it [00:29,  3.43it/s]


这一刻，萧炎的心中总算是轻松了不少，当然，因为薰儿的受伤，这对于他来说可是相当不利的，他压根不敢乱�


In [24]:
text = '''文本:我明天要去游乐园
时间:明天
文本:今天提醒我要出去吃饭完
时间:今天
文本:我好开心啊
时间:无
文本:能不能提醒我下周一要开会
时间:下周一
文本:我后天要去北京
时间:后天
文本:我周日要去吃海底捞
时间:'''
print(generate(text, max_len=100))

28it [00:07,  3.54it/s]

周日
文本:晚上要去邮电局



In [25]:
text = '''文本:今天提醒我要出去吃饭完
地点:无
文本:我好开心啊
地点:无
文本:能不能提醒我下周一要到杭州开会
地点:杭州
文本:我后天要去北京
地点:北京
文本:今天领导飞去郑州了
地点:'''
print(generate(text, max_len=100))

61it [00:17,  3.59it/s]

郑州
文本:千山万水
今天地震后
我想回到家乡
地点:嵩山

